In [ ]:
def simulate_trading(ticker, start_date, end_date_1,end_date_2,initial_capital):
    
    ticker = ticker
    start_date = start_date
    end_date1 = end_date_1
    finance_df,forecast = prophet_model(ticker, start_date, end_date_1)
    # 입력된 날짜 범위로 데이터 필터링
    filtered_df = finance_df[(finance_df['ds'] >= start_date) & (finance_df['ds'] <= end_date_1)]
    filtered_df_forecast = forecast[(forecast['ds'] >= start_date) & (forecast['ds'] <= end_date_2)]

    # 현재 자본 및 보유 주식 수 초기화
    current_capital = initial_capital
    shares_owned = 0

    # 현재 포지션 초기화
    position = 'None'

    # 실제 및 예측 주식 가격 설정
    actual_prices = filtered_df.set_index('ds')['y']
    predicted_prices = filtered_df_forecast.set_index('ds')['yhat']

    # 거래 기록 초기화
    trading_history = []

    # 시뮬레이션 시작
    for date, today_price, predicted_next_day_price in zip(actual_prices.index, actual_prices.values, predicted_prices.shift(-1).values):
        predicted_change = predicted_next_day_price - today_price

        # 매수 조건
        if predicted_change > 0 and position != 'Hold':
            shares_to_buy = current_capital / today_price
            shares_owned += shares_to_buy
            current_capital -= shares_to_buy * today_price
            position = 'Hold'
            trading_history.append((date, '매수', today_price, predicted_next_day_price))

        # 매도 조건
        elif predicted_change < 0 and position == 'Hold':
            current_capital += shares_owned * today_price
            shares_owned = 0
            position = 'None'
            trading_history.append((date, '매도', today_price, predicted_next_day_price))

        # 보유 조건
        elif position == 'Hold' and abs(predicted_change) <= 0.01 * today_price:
            trading_history.append((date, '보유', today_price, predicted_next_day_price))

        # 조치 없음 또는 보유
        else:
            if position == 'Hold':
                trading_history.append((date, '보유', today_price, predicted_next_day_price))
            else:
                trading_history.append((date, '조치 없음', today_price, predicted_next_day_price))

    # 최종 자본 및 수익률 계산
    final_capital = current_capital + (shares_owned * actual_prices.iloc[-1])
    profit = final_capital - initial_capital
    return_rate = (profit / initial_capital) * 100

    return trading_history, final_capital, return_rate

# 주어진 날짜 범위로 거래 시뮬레이션 실행
start_date = '2024-04-05'
end_date_1 = '2024-05-10'
end_date_2 = '2024-05-11'
initial_capital = 100000000.0
trading_history, final_capital, return_rate = simulate_trading(start_date, end_date_1,end_date_2,initial_capital)

# 거래 기록 출력
for record in trading_history:
    print(f"{record[0]} - {record[1]}: 현재 가격 ${record[2]:.3f}, 예측 다음날 가격 ${record[3]:.3f}")

# 최종 자본 및 수익률 출력
print(f"최종 자본: ${final_capital:.2f}")
print(f"수익률: {return_rate:.2f}%")
